# Introduction

Laboratory A needs to keep track of the clinical isolates data they receive from sequencing core in addition to their data that is being
generated everyday within the lab. Lack of a database has introduced data redundancy, inconsistency and many other issues such as difficulty
to retrieve and manipulate data in a timely manner. The process of business in this lab is as follows:

![Business Process](https://drive.google.com/uc?export=view&id=14_bqtaK1rrT4n6EphhKXFE13OJ2pXUY5)

## Advantages and contributions:
1. Reduce data redundancy
2. Faster ad-hoc queries
3. Reduce data inconsistency

## Uses cases:
Most computational labs or famous genomic/proteomic databases such as NCBI SRA, assembly or protein databases.

# The dataset

### Entity Relationship Diagram (ERD)
![ERD](https://drive.google.com/uc?export=view&id=1eWvJVvvrxeV0fILQGRlO1WQJ-7nuXsxX)

### Relational Diagram
![Relational Diagram](https://drive.google.com/uc?export=view&id=1fjwUNdw2U0wNW2pforsbWBR3DZLc1kaQ)


In [ ]:
import sys
!pip install pandas
!pip install numpy
!pip install datapane
!pip install plotly
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 kB 39.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.8 MB/s eta 0:00:00
  Created wheel for micawber: filename=micawber-0.5.5-py3-none-any.whl size=22434 sha256=56de5098339bfb0408d3b88cfc150a564210023ca419f31126c3539f6da678b0
  Stored in directory: /root/.cache/pip/wheels/31/3a/1f/bbd88c767423821f2fe497c0ada47581292f96419b56d689c0
Successfully built micawber
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not in

In [ ]:
# import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datapane as dp
import pymysql
import warnings
import pymysql
import sys
import os

warnings.filterwarnings('ignore')


In [ ]:
host = 'database-1.cmht6sfnmvcq.us-east-1.rds.amazonaws.com'
#port 3306
user = 'admin'
password = 'RLgrime!1'
database = 'final_schema' or 'final_schema2'

connection = pymysql.connect(host=host, user=user, passwd=password, database=database)
with connection:
    cur = connection.cursor()
    cur.execute("SELECT VERSION()")
    version = cur.fetchone()
    print("Database version: {} ".format(version[0]))

Database version: 8.0.33 


In [ ]:
def run_query(q: str) -> pd.DataFrame:
    with pymysql.connect(host=host, user=user, passwd=password, database=database) as conn:
        return pd.read_sql(q, conn)


def run_command(c: str):
    with pymysql.connect(host=host, user=user, passwd=password, database=database) as conn:
        conn.isolation_level = None
        conn.execute(c)

In [ ]:
def show_tables() -> pd.DataFrame:
    q = """
          SELECT table_name
          FROM information_schema.tables
          WHERE table_schema = 'final_schema' AND table_type = 'BASE TABLE';
        """
    return run_query(q)


def get_table_row_count(tablename) -> int:
    q = (
        """
        SELECT
            COUNT(1)
        FROM %s;
        """
        % tablename
    )
    return run_query(q)["COUNT(1)"][0]


tables = show_tables()
tables["row_count"] = [get_table_row_count(t) for t in tables['TABLE_NAME']]

tables

,TABLE_NAME,row_count
0,DST,50
1,DST_drug,82
2,drug,50
3,isolate,50
4,sequencing,50
5,treatment_length,50
6,treatments_drug,100


In [ ]:
top_countries = """
SELECT
    country,
    COUNT(country) as count_of_countries
FROM isolate
GROUP BY country
ORDER BY count_of_countries DESC, country
"""

run_query(top_countries)

,country,count_of_countries
0,Australia,3
1,Canada,3
2,China,3
3,France,3
4,Germany,3
5,India,3
6,Italy,3
7,Japan,3
8,Spain,3
9,USA,3


In [ ]:
country_metrics = run_query(top_countries)

country_breakdown = go.Figure(
    data=[go.Pie(labels=country_metrics["country"], values=country_metrics["count_of_countries"], hole=0.3)]
)

country_breakdown.update_layout(title_text="Number of Samples by Country")

country_breakdown.show()

In [ ]:
demographics = """

SELECT
    SUM(CASE WHEN HIV = 1 THEN 1 ELSE 0 END) AS count_of_HIV_TRUE,
    SUM(CASE WHEN diabetes = 1 THEN 1 ELSE 0 END) AS count_of_diabetes_TRUE,
    SUM(CASE WHEN smoker = 1 THEN 1 ELSE 0 END) AS count_of_smoker_TRUE,
    SUM(CASE WHEN HIV = 0 THEN 1 ELSE 0 END) AS count_of_HIV_FALSE,
    SUM(CASE WHEN diabetes = 0 THEN 1 ELSE 0 END) AS count_of_diabetes_FALSE,
    SUM(CASE WHEN smoker = 0 THEN 1 ELSE 0 END) AS count_of_smoker_FALSE
FROM isolate;
"""

run_query(demographics)

,count_of_HIV_TRUE,count_of_diabetes_TRUE,count_of_smoker_TRUE,count_of_HIV_FALSE,count_of_diabetes_FALSE,count_of_smoker_FALSE
0,25.0,25.0,29.0,25.0,25.0,21.0


In [ ]:
demo = run_query(demographics)
labels = demo.columns
values = demo.iloc[0]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=["HIV Positive","Diabetic","Smoker"],
    y=values[:3],
    name='TRUE',
    marker_color='blue'
))
fig.add_trace(go.Bar(
    x=["HIV Positive","Diabetic","Smoker"],
    y=values[3:],
    name='FALSE',
    marker_color='red'
))

fig.update_layout(barmode='stack', title_text='Demographics of Isolate Donors', title_x=0.5)

fig.show()

In [ ]:
drugs2 = """
SELECT
    drug_name,
    AVG(treatment_length) as average_treatment_length,
    SUM(CASE WHEN DST.susceptibility = 1 THEN 1 ELSE 0 END) AS count_susceptible,
    SUM(CASE WHEN DST.susceptibility = 0 THEN 1 ELSE 0 END) AS count_not_susceptible
FROM drug
JOIN DST_drug ON drug.drug_id = DST_drug.drug_ID
JOIN DST ON DST_drug.DST_ID = DST.DST_ID
LEFT JOIN (
    SELECT drug_id, treatment_ID
    FROM treatments_drug
    GROUP BY drug_id
) AS td_agg ON drug.drug_id = td_agg.drug_id
JOIN treatment_length ON td_agg.treatment_ID = treatment_length.treatment_ID
GROUP BY drug.drug_name
ORDER BY drug_name

"""
run_query(drugs2)

,drug_name,average_treatment_length,count_susceptible,count_not_susceptible
0,Amikacin,10.0,2.0,0.0
1,Amoxicillin,14.0,2.0,0.0
2,Azithromycin,21.0,0.0,2.0
3,Aztreonam,21.0,0.0,2.0
4,Cefazolin,10.0,2.0,0.0
5,Cefepime,14.0,2.0,0.0
6,Cefotaxime,28.0,0.0,2.0
7,Cefoxitin,14.0,2.0,0.0
8,Ceftazidime,28.0,0.0,2.0
9,Ceftriaxone,21.0,0.0,4.0


In [ ]:
from plotly.subplots import make_subplots

drug_data2 = run_query(drugs2)
drug_names = drug_data2['drug_name']
average_treatment_length = drug_data2['average_treatment_length']
count_susceptible = drug_data2['count_susceptible']
count_not_susceptible = drug_data2['count_not_susceptible']

fig2 = make_subplots(specs=[[{'secondary_y': True}]])

fig2.add_trace(go.Bar(x=drug_names, y=count_susceptible, name='Count Susceptible', marker_color='blue'), secondary_y=False)
fig2.add_trace(go.Bar(x=drug_names, y=count_not_susceptible, name='Count Not Susceptible', marker_color='orange'), secondary_y=False)

fig2.add_trace(go.Scatter(x=drug_names, y=average_treatment_length, mode='lines+markers', name='Average Treatment Length', marker=dict(color='green')), secondary_y=True)

fig2.update_layout(title_text='Drug Susceptibility and Average Treatment Length', title_x=0.5)

In [ ]:
sample_types = """
SELECT sample_type, COUNT(sample_type) as count_of_sample_types
from isolate
GROUP BY sample_type
"""

run_query(sample_types)

,sample_type,count_of_sample_types
0,Blood,5
1,Sputum,6
2,Urine,5
3,CSF,5
4,Wound swab,5
5,Stool,5
6,Nasal swab,5
7,Tissue biopsy,5
8,Joint fluid,5
9,Throat swab,4


In [ ]:
sample_type_counts = run_query(sample_types)

samples = go.Figure(
    data=[go.Pie(labels=sample_type_counts["sample_type"], values=sample_type_counts["count_of_sample_types"], hole=0.3)]
)

samples.update_layout(title_text="Sample Types of Isolates")

samples.show()

In [1]:
r = dp.View(
    dp.Plot(country_breakdown, name="genre_sales"),
    dp.Plot(samples),
    dp.Plot(fig),
    dp.Plot(fig2),
)

dp.save_report(r, "report.html", open=True)

NameError: ignored

In [ ]:
import IPython
IPython.display.HTML(filename='report.html')